In [21]:
import requests
import numpy as np
import pandas as pd
import re
import key

pd.set_option('display.max_columns', None)

# Get Anime Data from API

In [22]:
# api key for the header
header = {'X-MAL-CLIENT-ID':key.api_key}

In [23]:
# example
anime_response = requests.get('https://api.myanimelist.net/v2/anime/ranking?ranking_type=bypopularity&limit=2&fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,start_season,broadcast,source,average_episode_duration,rating,studios',  headers=header)
anime_response_json = anime_response.json()
anime_response_json

{'data': [{'node': {'id': 16498,
    'title': 'Shingeki no Kyojin',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/10/47347.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/10/47347l.jpg'},
    'alternative_titles': {'synonyms': ['AoT', 'SnK'],
     'en': 'Attack on Titan',
     'ja': '進撃の巨人'},
    'start_date': '2013-04-07',
    'end_date': '2013-09-29',
    'synopsis': 'Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called Titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. To ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. However, that fragile calm is soon shattered when a colossal Titan manages to breach the supposedly impregnab

In [24]:
# fetch the top 10K most popular anime
x = 0
anime_list = []

api_url = 'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=500&fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,start_season,broadcast,source,average_episode_duration,rating,studios'

while x < 10000:
    anime_response_json = requests.get(api_url, headers=header).json()
    for i in range(500):
        anime_list.append(anime_response_json['data'][i])

    api_url = anime_response_json['paging']['next']
    x += 500

print('anime_list length:', len(anime_list))

anime_list length: 10000


In [25]:
df = pd.DataFrame(anime_list)
df.head()

,node,ranking
0,"{'id': 5114, 'title': 'Fullmetal Alchemist: Br...",{'rank': 1}
1,"{'id': 9253, 'title': 'Steins;Gate', 'main_pic...",{'rank': 2}
2,"{'id': 51535, 'title': 'Shingeki no Kyojin: Th...",{'rank': 3}
3,"{'id': 41467, 'title': 'Bleach: Sennen Kessen-...",{'rank': 4}
4,"{'id': 28977, 'title': 'Gintama°', 'main_pictu...",{'rank': 5}


In [26]:
# normalize
normalized_df = pd.json_normalize(df['node'])
normalized_df.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,visual_novel,1460,pg_13,"[{'id': 314, 'name': 'White Fox'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN


# Clean up the Genres Column

In [27]:
normalized_df['genres'][0]

[{'id': 1, 'name': 'Action'},
 {'id': 2, 'name': 'Adventure'},
 {'id': 8, 'name': 'Drama'},
 {'id': 10, 'name': 'Fantasy'},
 {'id': 38, 'name': 'Military'},
 {'id': 27, 'name': 'Shounen'}]

In [28]:
# normalize the genres column
normalized_df2 = normalized_df.join(pd.json_normalize(normalized_df.pop('genres')))
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,0,1,2,3,4,5,6,7,8,9,10,11
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"{'id': 1, 'name': 'Action'}","{'id': 2, 'name': 'Adventure'}","{'id': 8, 'name': 'Drama'}","{'id': 10, 'name': 'Fantasy'}","{'id': 38, 'name': 'Military'}","{'id': 27, 'name': 'Shounen'}",None,None,None,None,None,None
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,"[{'id': 314, 'name': 'White Fox'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,"{'id': 8, 'name': 'Drama'}","{'id': 40, 'name': 'Psychological'}","{'id': 24, 'name': 'Sci-Fi'}","{'id': 41, 'name': 'Suspense'}","{'id': 78, 'name': 'Time Travel'}",None,None,None,None,None,None,None
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"{'id': 1, 'name': 'Action'}","{'id': 8, 'name': 'Drama'}","{'id': 58, 'name': 'Gore'}","{'id': 38, 'name': 'Military'}","{'id': 27, 'name': 'Shounen'}","{'id': 76, 'name': 'Survival'}","{'id': 41, 'name': 'Suspense'}",None,None,None,None,None


In [29]:
# get rid of unnecessary characters in the genres columns
for col in range(0,12):
    normalized_df2[col] = normalized_df2[col].apply(lambda x: re.sub('[{}\'0-9:,]', '', str(x)))
    normalized_df2[col] = normalized_df2[col].apply(lambda x: x.replace('id', '').replace('name', '').replace('None', '').strip())
    
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,0,1,2,3,4,5,6,7,8,9,10,11
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,Action,Adventure,Drama,Fantasy,Military,Shounen,,,,,,
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,"[{'id': 314, 'name': 'White Fox'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,Drama,Psychological,Sci-Fi,Suspense,Time Travel,,,,,,,
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,Action,Drama,Gore,Military,Shounen,Survival,Suspense,,,,,


In [30]:
# get integer location of the column 0
normalized_df2.columns.get_loc(0)

29

In [31]:
# combine genres
normalized_df2['genres'] = normalized_df2[normalized_df2.columns[29:]].agg(' '.join, axis=1)
normalized_df2['genres'] = normalized_df2['genres'].apply(lambda x: ', '.join([word for word in x.split() if word != '']))

In [32]:
normalized_df2['genres'][0]

'Action, Adventure, Drama, Fantasy, Military, Shounen'

In [33]:
# drop the normalized columns (0 to 11)
normalized_df2.drop([0,1,2,3,4,5,6,7,8,9,10,11], axis=1, inplace=True)
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S..."
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,"[{'id': 314, 'name': 'White Fox'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ..."
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv..."


# Do the Same for the Studios Column

In [34]:
# normalize the studios column
normalized_df3 = normalized_df2.join(pd.json_normalize(normalized_df2.pop('studios')))
normalized_df3.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,0,1,2,3,4,5,6,7,8,9
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...","{'id': 4, 'name': 'Bones'}",None,None,None,None,None,None,None,None,None
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","{'id': 314, 'name': 'White Fox'}",None,None,None,None,None,None,None,None,None
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv...","{'id': 569, 'name': 'MAPPA'}",None,None,None,None,None,None,None,None,None


In [35]:
for col in range(0,10):
    normalized_df3[col] = normalized_df3[col].apply(lambda x: re.sub('[{}\'0-9:,]', '', str(x)))
    normalized_df3[col] = normalized_df3[col].apply(lambda x: x.replace('id', '').replace('name', '').replace('None', '').strip())

In [36]:
# get integer location of the column 0
normalized_df3.columns.get_loc(0)

29

In [37]:
# combine studios
normalized_df3['studios'] = normalized_df3[normalized_df3.columns[29:]].agg(' '.join, axis=1)
normalized_df3['studios'] = normalized_df3['studios'].apply(lambda x: ', '.join([word for word in x.split() if word != '']))

In [38]:
# drop the normalized columns (0 to 9)
normalized_df3.drop([0,1,2,3,4,5,6,7,8,9], axis=1, inplace=True)
normalized_df3.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3152647,2005487,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.08,2,13,2421413,1326575,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox"
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.07,3,508,411436,141085,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv...",MAPPA


In [39]:
normalized_df3.shape

(10000, 30)

In [40]:
normalized_df3.to_csv('data/anime_data.csv', index=False)